In [1]:
import cv2
import os
import numpy as np
import pyautogui as pauto

In [2]:
facecascPath=os.path.dirname(cv2.__file__)+"/data/haarcascade_frontalface_default.xml"
eyecascPath=os.path.dirname(cv2.__file__)+"/data/haarcascade_eye.xml"
faceCascade = cv2.CascadeClassifier(facecascPath)
eye_cascade = cv2.CascadeClassifier(eyecascPath)

In [3]:
def detect_faces(img, cascade):
    gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    coords = cascade.detectMultiScale(gray_frame,scaleFactor=1.1,minNeighbors=5,flags=cv2.CASCADE_SCALE_IMAGE)
    if len(coords) > 1:
        biggest = (0, 0, 0, 0)
        for i in coords:
            if i[3] > biggest[3]:
                biggest = i
        biggest = np.array([i], np.int32)
    elif len(coords) == 1:
        biggest = coords
    else:
        return None
    return biggest

In [4]:
def detect_eyes(img, cascade):
    gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    eyes = cascade.detectMultiScale(gray_frame,scaleFactor=1.1,minNeighbors=5,flags=cv2.CASCADE_SCALE_IMAGE) # detect eyes
    width = np.size(img, 1) # get face frame width
    height = np.size(img, 0) # get face frame height
    left,right = (None,None)
    for (x, y, w, h) in eyes:
        if y > height / 2:
            pass
        eyecenter = x + w / 2  # get the eye center
        if eyecenter < width * 0.5:
            left = (y,y + h, x,x + w)
        else:
            right = (y,y + h, x,x + w)
    return left, right

In [5]:
def cut_eyebrows(img):
    height, width = img.shape[:2]
    eyebrow_h = int(height / 4)
    img = img[eyebrow_h:height, 0:width]
    return img

In [6]:
def blob_process(img, detector):
    keypoints = detector.detect(img)
    return keypoints

In [7]:
def mouse_index(x,y):
    return (x,y)


In [ ]:
video_capture = cv2.VideoCapture(0)

rows,cols = pauto.size()
pauto.moveTo(rows/2,cols/2)
while True:
    ret, img = video_capture.read()
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rows_i,cols_i,_ = img.shape
    x_mul = (int)(rows/rows_i)
    y_mul = (int)(cols/cols_i)
    
    faces = detect_faces(img,faceCascade)
    
    try:
        for (p,q,r,s) in faces:
#             cv2.rectangle(img,(p,q),(p+r,q+s),(255,0,0),3)
            x_mean = (p+(p+r))/2
            y_mean = (q+(q+s))/2
            
            face_gray = gray_img[q:q+s, p:p+r]
            face_color = img[q:q+s, p:p+r]
            
#             cv2.line(img, (int(x_mean), int(0)), (int(x_mean), int(rows)), (0, 255, 0), 2)
#             cv2.line(img, (int(0),int(y_mean)), (int(cols),int(y_mean)), (0, 255, 0), 2)
            
            mouse = mouse_index(int(x_mean),int(y_mean))
            cv2.circle(img,mouse, radius=5, color=(127,0,0), thickness=-1)            
            
            (left_eye,right_eye) = detect_eyes(face_color,eye_cascade)
            if(right_eye!=None):
                (hx,hq,hr,hs) = right_eye
                eye_gray = face_gray[hx:hq,hr:hs]
                eye_roi = cv2.GaussianBlur(eye_gray, (7, 7), 0)
                eye_color = face_color[hx:hq,hr:hs]
                
                _, threshold = cv2.threshold(eye_roi, 35, 255, cv2.THRESH_BINARY_INV)
                threshold = cut_eyebrows(threshold)
            
                contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)
            
                for cnt in contours:
                    (xu, yu, wu, hu) = cv2.boundingRect(cnt)
                    
#                     cv2.rectangle(eye_color, (xu, yu), (xu + wu, yu + hu), (255, 0, 0), 2)
                    eye_xmean = (int)((2*xu+wu)/2)
                    eye_ymean = (int)((2*yu+hu)/2)
                    cv2.circle(eye_color,(eye_xmean,eye_ymean), radius=5, color=(100,0,0), thickness=-1)
                    
                    a,b = (eye_xmean,eye_ymean)
                    pauto.moveTo((a*x_mul),(b*y_mul))
                    break
                            
            elif(left_eye!=None):
                (hx,hq,hr,hs) = left_eye
                eye_gray = face_gray[hx:hq,hr:hs]
                eye_roi = cv2.GaussianBlur(eye_gray, (7, 7), 0)
                eye_color = face_color[hx:hq,hr:hs]
                
                _, threshold = cv2.threshold(eye_roi, 35, 255, cv2.THRESH_BINARY_INV)
                threshold = cut_eyebrows(threshold)
                
            
                contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)
            
                for cnt in contours:
                    (xu, yu, wu, hu) = cv2.boundingRect(cnt) 
#                     cv2.rectangle(eye_color, (xu, yu), (xu + wu, yu + hu), (255, 0, 0), 2)
                    eye_xmean = (int)((2*xu+wu)/2)
                    eye_ymean = (int)((2*yu+hu)/2)
                    cv2.circle(eye_color,(eye_xmean,eye_ymean), radius=5, color=(100,0,0), thickness=-1)
                    a,b = (eye_xmean,eye_ymean)
                    pauto.moveTo((a*x_mul),(b*y_mul))
                    break
            else:
                print('no eyes')
    except:
        print('no face')

    rimg = cv2.flip(img, 1) # invert the object img
    cv2.imshow("Video", rimg)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no eyes
no eyes
no eyes
no face
no face
no face
no face
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no face
no face
no face
no face
no face
no face
no face
no face
no face
no eyes
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no eyes
no eyes
no face
no face
no face
no eyes
no eyes
no face
no eyes
no eyes
no eyes
no face
no face
no face
no face
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no face
no face
no eyes
no eyes
no face
no face
no face
no face
no face
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes
no eyes


In [ ]:
video_capture.release()
cv2.destroyAllWindows()